In [11]:
import numpy as np
import random
import xgboost as xgb

In [12]:
# import the xgboost model
model = xgb.Booster()
model.load_model('models/baseline.xgb')


In [13]:
def predict_movement(bid, ask):
    dtest = xgb.DMatrix(np.array([[bid, ask]]))
    prediction = model.predict(dtest)
    return prediction[0]  # Probability of price going up


In [14]:
def trade_simulation(strategy, data):
    cash = 1000  # Starting cash
    holdings = 0  # No holdings initially
    transaction_cost = 0.01  # Cost per transaction as a percentage

    for bid, ask in data:
        prediction = predict_movement(bid, ask)
        
        if prediction > strategy[0] and cash > 0:  # Buy condition
            holdings += (cash / ask) * (1 - transaction_cost)
            cash = 0
        elif prediction < strategy[1] and holdings > 0:  # Sell condition
            cash += holdings * bid * (1 - transaction_cost)
            holdings = 0

    # Final asset conversion to cash
    final_cash = cash + (holdings * bid if holdings > 0 else 0)
    return final_cash - 1000  # profit or loss from initial cash

In [15]:
import random
from deap import base, creator, tools, algorithms

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("attribute", random.uniform, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attribute, n=2)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# toolbox.register("evaluate", lambda ind: (trade_simulation(ind, market_data),))
toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=0.1, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)